# Call Center variant

This notebook has a variation of the `telco_callcenter` example, including usage of a `Sequence` to chain `Team` output with a "filter" that adjusts the response given the channel the conversation is happening on.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Add the parent directory to sys.path
import sys, os
sys.path.append(os.path.abspath(os.path.join('../vanilla_aiagents')))

from vanilla_aiagents.agent import Agent
from vanilla_aiagents.user import User
from vanilla_aiagents.workflow import Workflow
from vanilla_aiagents.team import Team
from vanilla_aiagents.sequence import Sequence
from vanilla_aiagents.llm import AzureOpenAILLM
from dotenv import load_dotenv

load_dotenv(override=True)
import os

In [3]:
llm = AzureOpenAILLM({
    "azure_deployment": os.getenv("AZURE_OPENAI_MODEL"),
    "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.getenv("AZURE_OPENAI_KEY"),
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
})

# Set logging to debug for Agent, User and Workflow
import logging

# Set logging to debug for Agent, User, and Workflow
logging.basicConfig(level=logging.WARN)
logging.getLogger("vanilla_aiagents.agent").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.team").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.user").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.workflow").setLevel(logging.DEBUG)

In [4]:
user = User(mode="unattended", description = "An interface to a human participant. Call this agent to get inputs from the user and proceed with the workflow.")

In [ ]:
greeter = Agent(description="A greeter agent, to say hello and welcome to the user", id="greeter", llm=llm, system_message="""
You are a call center operator that responds to customer inquiries. 
    
    Your task are:
    - Greet the Customer at first. Be sure to ask how you can help.
    - Check if the Customer has any additional questions. If not, close the conversation.
    - Close the conversation after the Customer's request has been resolved. Thank the Customer for their time and wish them a good day and write TERMINATE to end the conversation. DO write TERMINATE in the response.
    
    IMPORTANT NOTES:
    - Make sure to act politely and professionally.    
    - Make sure to write TERMINATE to end the conversation.    
    - NEVER pretend to act on behalf of the company. NEVER provide false information.
 """)

In [ ]:
sales = Agent(description="A sales agent that can answer sales questions", id="sales",llm=llm, system_message="""
You are a sales person that responds to customer inquiries.
    
    You have access to pricing and product details in the PRODUCTS sections below. Please note field starting with "_" are not to be shared with the Customer.
    
    Your tasks are:
    - provide the Customer with the information they need. Try to be specific and provide the customer only options that fit their needs.
    
    IMPORTANT NOTES:
    - DO act politely and professionally
    - NEVER provide false information
    
    ### PRODUCTS
    - Mobile Internet
        - Description: Mobile WiFi for you to take anywhere, supports up to 10 devices.
        - Price: €10/month
        - Details: 10GB data included, €1/GB after that.
        - _SKU: INET_MOBILE
    - All-in-One Bundle
        - Description: Mobile internet and home internet in one package.
        - Price: €45/month
        - Details: 10GB mobile data, €1/GB after that. Home internet included.
        - _SKU: INET_BUNDLE
    - Home Internet
        - Description: High-speed internet for your home.
        - Price: €30/month
        - Details: Unlimited data at 1Gbps.
        - _SKU: INET_HOME""")

In [ ]:
from typing_extensions import Annotated


technical = Agent(description="A technical support agent that can answer technical questions", id="technical", llm=llm, system_message="""You are a technical support agent that responds to customer inquiries.
    
    Your task are:
    - Assess the technical issue the customer is facing.
    - Verify if there any known issues with the service the customer is using.
    - Check remote telemetry data to identify potential issues with customer's device. Be sure to ask customer code first.
    - Provide the customer with possible solutions to the issue. See the list of common issues below.
    - When the service status is OK, reply the customer and suggest to restart the device.
    - When the service status is DEGRADED, apologize to the customer and kindly ask them to wait for the issue to be resolved.
    - Open an internal ticket if the issue cannot be resolved immediately.
    
    Make sure to act politely and professionally.    
    
    ### Common issues and solutions:

    - Home Internet:
        - Issue: No internet connection.
        - Solutions: 
            - Check the router's power supply and cables.
            - Restart the router.
            - Check the internet connection status LED on the router.
    - Mobile Internet:
        - Issue: Slow internet connection or no connection.
        - Solutions:
            - Check the signal strength on the device.
            - Restart the device.
            - Check the data usage on the device.
            - Suggest the customer to purchase additional data when the limit is reached.
    - All-in-One Bundle:
        USE a combination of the solutions for Home Internet and Mobile Internet.
    
    """)

@technical.register_tool(description="Get the service status")
def get_service_status(
    service_sku: Annotated[str, "The SKU of the service to check status for"]
    ) -> Annotated[str, "Status of the specified service"]:
        
    return "Service degraded"

@technical.register_tool(description="Get the service status")
def check_customer_telemetry(
    service_sku: Annotated[str, "The SKU of the service to check status for, values can be INET_MOBILE, INET_BUNDLE, INET_HOME"],
    customerCode: Annotated[str, "The customer code to check telemetry for"]
    ) -> Annotated[str, "Telemetry summary for the specified customer"]:
            
    return "No issues detected"

In [ ]:
team = Team(id="team", description="", members=[user, greeter, sales, technical], llm=llm, stop_callback=lambda msgs: len(msgs) > 6)


In [ ]:
channel = Agent(description="An agent that detects the channel of the conversation", id="channel", llm=llm, system_message="""
You are a channel detection agent that determines the channel of the conversation.
Your task is to write the channel of the LAST USER message. The channel can be one of the following value:
- Voice
- Web
- WhatsApp
You MUST only output CHANNEL= followed by the channel value.

NOTE: since this is a demo, the channel MUST ALWAYS BE 'Voice'.
""")

In [ ]:
reviewer = Agent(description="A reviewer agent that MUST review the conversation and adjust other agent messages", 
                 id="reviewer", 
                 llm=llm,
                 system_message="""
You are a reviewer that reviews messages from other agents and adjusts them as needed.

YOUR TASKS:
- Find the message from 'content-provider' to understand the context information.
- Find the last message in the conversation from an assistant. MAKE sure to ignore user messages.
- If the channel is 'Voice', REWRITE the message to sound more natural, be shorter and does not sound like someone reading a pitch.
- If the channel is 'WhatsApp', REWRITE the message to remove any Markdown formatting.
- ALWAYS ensure the same information is conveyed in the edited message.
- NEVER edit the message from the user
- Make sure all edits keep the message polite and professional
- NEVER provide false information or pretend to act on behalf of the user
""")

In [11]:
from vanilla_aiagents.askable import Askable
from vanilla_aiagents.llm import LLM


class ReviewerFilter(Askable):
    def __init__(self, id: str, description: str, llm: LLM):
        super().__init__(id, description)
        self.llm = llm

    def ask(self, messages: list[dict]):
        # Find the message from name=context-provider
        context_provider_message = next((message for message in messages if message["name"] == "context-provider"), None)
        context = context_provider_message["content"]
        last_message = messages[-1]
        
        system_message_template = """
You are a reviewer that reviews messages from other agents and adjusts them as needed.

YOUR TASKS:
- Use the context information to understand the message.
- If the channel is 'Voice', REWRITE the message to sound more natural, be shorter and does not sound like someone reading a pitch. Also strip any Markdown formatting.
- If the channel is 'WhatsApp', REWRITE the message to remove any Markdown formatting.
- ALWAYS ensure the same information is conveyed in the edited message.
- Make sure all edits keep the message polite and professional
- NEVER provide false information or pretend to act on behalf of the user

CONTEXT INFORMATION:
{context}

MESSAGE TO REVIEW:
{message}
        """
        
        local_messages = []
        local_messages.append({"role": "system", "content": system_message_template.format(context=context, message=last_message["content"])})
        local_messages.append({"role": "user", "content": "Rewrite the message"})
        
        response = self.llm.ask(messages=local_messages)
        
        messages[-1]["content"] = response.content
        
        return None

In [ ]:
filter = ReviewerFilter(id="filter", description="A reviewer filter", llm=llm)
flow = Sequence(id="flow", llm=llm, description="A flow to handle customer inquiries", steps=[team, filter])

In [ ]:
history = [
    { "role": "assistant", "name": "context-provider", "content": "CHANNEL=voice" },
]
workflow = Workflow(askable=flow, messages=history)
# workflow.restart()
workflow.run("Hi")
workflow.messages

In [ ]:
workflow.run("Hi, I want to subscribe a new Internet plan for my home. Can you help me with that?")
# workflow.run("I have problems with my home internet")
workflow.messages

In [ ]:
workflow.run("I have no connection")
workflow.messages[-1]

In [ ]:
workflow.run("My customer code is 1234")
workflow.messages[-1]

In [ ]:
for msg in workflow.messages:
    if 'name' not in msg:
        msg['name'] = msg['role']
    print(f"{msg['role']}\t{msg['name']}\t'{msg['content']}'")